In [1]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.8 MB/s eta 0:00:00


In [2]:
# Deutsch Algorithm using Qiskit 2.x
# Compatible with Qiskit 2.0+ (2024–2025)

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# Choose the function type: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'
function_type = 'balanced_1'

def deutsch_oracle(qc, function_type):
    """Implements oracle Uf for given function type."""
    if function_type == 'constant_0':
        # f(x)=0 → Do nothing
        pass
    elif function_type == 'constant_1':
        # f(x)=1 → Apply X on the output qubit
        qc.x(1)
    elif function_type == 'balanced_0':
        # f(x)=x → Apply CNOT (control: input, target: output)
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        # f(x)=NOT(x) → Apply X, then CNOT, then X
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)

In [15]:
# Step 1: Initialize quantum circuit with 2 qubits and 1 classical bit
qc = QuantumCircuit(2, 1)

# Step 2: Initialize |x>|y> = |0>|1> and apply Hadamard
qc.x(1)             # Set output qubit to |1>
qc.barrier()
qc.h([0, 1])        # Apply Hadamard to both qubits

# Step 3: Apply the oracle
qc.barrier()
qc.compose(deutsch_oracle(function_type), inplace=True)

In [4]:
# Step 4: Apply Hadamard to input qubit
qc.barrier()
qc.h(0)

# Step 5: Measure the first qubit
qc.measure(0, 0)

# Visualize circuit
print(qc.draw(output="text"))

           ░ ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐┌─┐
q_0: ──────░─┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├┤M├
     ┌───┐ ░ ├───┤ ░ └───┘┌─┴─┐└───┘ ░ └───┘└╥┘
q_1: ┤ X ├─░─┤ H ├─░──────┤ X ├──────░───────╫─
     └───┘ ░ └───┘ ░      └───┘      ░       ║ 
c: 1/════════════════════════════════════════╩═
                                             0 


In [5]:
# Step 6: Simulate
sim = AerSimulator()
qc_compiled = transpile(qc, sim)
result = sim.run(qc_compiled).result()
counts = result.get_counts()

print("\nMeasurement results:", counts)

# Interpret result
if list(counts.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")


Measurement results: {'1': 1024}
→ Function is BALANCED.


In [11]:
!pip install qiskit qiskit-aer --quiet


In [13]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt

In [14]:
# function_type ∈ {'constant_0', 'constant_1', 'balanced_0', 'balanced_1'}
def deutsch_oracle(function_type: str) -> QuantumCircuit:
    qc = QuantumCircuit(2, name=f"oracle_{function_type}")
    if function_type == 'constant_0':
        pass
    elif function_type == 'constant_1':
        qc.x(1)
    elif function_type == 'balanced_0':
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        qc.cx(0, 1)
        qc.x(1)
    else:
        raise ValueError("Invalid function_type")
    return qc

In [16]:
def deutsch_circuit(function_type: str):
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(qr, cr)

    # Prepare |1> in ancilla and superposition
    qc.x(1)
    qc.h(0)
    qc.h(1)

    # Oracle
    oracle = deutsch_oracle(function_type)
    qc.append(oracle.to_instruction(), [0, 1])

    # Interference
    qc.h(0)
    qc.measure(0, 0)

    return qc


In [28]:
def run_deutsch(function_type, shots=1024, use_statevector=False, noise_model=None):
    qc = deutsch_circuit(function_type)
    simulator = AerSimulator()

    if use_statevector:
        sv_qc = qc.remove_final_measurements(inplace=False)
        sv_sim = AerSimulator(method='statevector')
        result_sv = sv_sim.run(transpile(sv_qc, sv_sim)).result()
        state = result_sv.get_statevector(sv_qc)
        plot_bloch_multivector(state)
        plt.show()

    if noise_model:
        t_qc = transpile(qc, simulator)
        job = simulator.run(t_qc, noise_model=noise_model, shots=shots)
    else:
        job = simulator.run(transpile(qc, simulator), shots=shots)

    result = job.result()
    counts = result.get_counts()
    bit = max(counts, key=counts.get)
    classification = "CONSTANT" if bit == '0' else "BALANCED"

    print(f"Function Type: {function_type}")
    print("Counts:", counts)
    print("Most frequent measurement:", bit)
    print("Result:", classification)

    # Display circuit using text output
    print("\nCircuit Diagram (text):")
    print(qc.draw(output="text", idle_wires=False))

In [29]:
!pip install pylatexenc
for f in ['constant_0', 'constant_1', 'balanced_0', 'balanced_1']:
    run_deutsch(f)
    print('-'*40)


Function Type: constant_0
Counts: {'0': 1024}
Most frequent measurement: 0
Result: CONSTANT

Circuit Diagram (text):
     ┌───┐     ┌────────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                   ├┤ H ├┤M├
     ├───┤┌───┐│  oracle_constant_0 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                   ├──────╫─
     └───┘└───┘└────────────────────┘      ║ 
c: 1/══════════════════════════════════════╩═
                                           0 
----------------------------------------
Function Type: constant_1
Counts: {'0': 1024}
Most frequent measurement: 0
Result: CONSTANT

Circuit Diagram (text):
     ┌───┐     ┌────────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                   ├┤ H ├┤M├
     ├───┤┌───┐│  oracle_constant_1 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                   ├──────╫─
     └───┘└───┘└────────────────────┘      ║ 
c: 1/══════════════════════════════════════╩═
                                           0 
----------------------------------------
Function Type: balanced_0
Counts: {'1': 

In [31]:
#Custom oracle
def oracle_x_xor_1():
    qc = QuantumCircuit(2, name="oracle_x_xor_1")
    qc.cx(0, 1)
    qc.x(1)
    return qc

def deutsch_custom(oracle):
    qc = QuantumCircuit(2, 1)
    qc.x(1)
    qc.h(0)
    qc.h(1)
    qc.append(oracle.to_instruction(), [0, 1])
    qc.h(0)
    qc.measure(0, 0)
    return qc

custom_oracle = oracle_x_xor_1()
qc_custom = deutsch_custom(custom_oracle)
sim = AerSimulator()
result = sim.run(transpile(qc_custom, sim), shots=1024).result()
counts = result.get_counts()
print("\nCircuit Diagram (text):")
print(qc_custom.draw('text'))
print("Custom oracle counts:", counts)
print("→ Expected: BALANCED (measurement 1)")


Circuit Diagram (text):
     ┌───┐     ┌─────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                ├┤ H ├┤M├
     ├───┤┌───┐│  oracle_x_xor_1 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                ├──────╫─
     └───┘└───┘└─────────────────┘      ║ 
c: 1/═══════════════════════════════════╩═
                                        0 
Custom oracle counts: {'1': 1024}
→ Expected: BALANCED (measurement 1)


In [32]:
error_1 = depolarizing_error(0.02, 1)
error_2 = depolarizing_error(0.02, 2)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

print("Running with noise model (2% depolarizing error)...")
run_deutsch('balanced_0', noise_model=noise_model)


Running with noise model (2% depolarizing error)...
Function Type: balanced_0
Counts: {'0': 35, '1': 989}
Most frequent measurement: 1
Result: BALANCED

Circuit Diagram (text):
     ┌───┐     ┌────────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                   ├┤ H ├┤M├
     ├───┤┌───┐│  oracle_balanced_0 │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                   ├──────╫─
     └───┘└───┘└────────────────────┘      ║ 
c: 1/══════════════════════════════════════╩═
                                           0 
